# Part D: Increase the number of hidden layers
## Repeat Part B but use 100 epochs this time for training
## Repeat part B but use a neural network with the following instead:

### - Three hidden layers, each of 10 nodes and ReLU activation function.

In [1]:
# download the concrete_data.csv file from url directly
! wget -O concrete_data.csv https://cocl.us/concrete_data

--2024-03-15 16:26:39--  https://cocl.us/concrete_data
Resolving cocl.us (cocl.us)... 2600:140f:5::173f:6dc3, 2600:140f:5::173f:6db8, 23.217.111.73, ...
Connecting to cocl.us (cocl.us)|2600:140f:5::173f:6dc3|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv [following]
--2024-03-15 16:26:40--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58988 (58K) [text/csv]
Saving to: ‘concrete_data.csv’

concrete_data.csv   100%[===================>]  57.61K  35.4KB/s    in

## Let's import all necessary modules


In [2]:
import pandas as pd    # for data analysis
from sklearn.model_selection import train_test_split   #for splitting the data into training and testing sets
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_squared_error    # for calculating mean_square error

2024-03-15 16:26:47.183726: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 16:26:47.260013: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 16:26:47.260157: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 16:26:47.260243: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-15 16:26:47.272886: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-15 16:26:47.273352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

## Let's read the data from csv file and analyzing it using pandas

In [3]:
df = pd.read_csv("concrete_data.csv")    # reading the data from csv file

In [4]:
df.shape  # it has 1030 rows and 9 columns

(1030, 9)

### let's read the top 10 rows of the data

In [5]:
df.head(n=10)

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
5,266.0,114.0,0.0,228.0,0.0,932.0,670.0,90,47.03
6,380.0,95.0,0.0,228.0,0.0,932.0,594.0,365,43.70
7,380.0,95.0,0.0,228.0,0.0,932.0,594.0,28,36.45
8,266.0,114.0,0.0,228.0,0.0,932.0,670.0,28,45.85
9,475.0,0.0,0.0,228.0,0.0,932.0,594.0,28,39.29


### Statistics summary of the data

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Cement,1030.0,281.167864,104.506364,102.00,192.375,272.900,350.000,540.0
Blast Furnace Slag,1030.0,73.895825,86.279342,0.00,0.000,22.000,142.950,359.4
Fly Ash,1030.0,54.188350,63.997004,0.00,0.000,0.000,118.300,200.1
Water,1030.0,181.567282,21.354219,121.80,164.900,185.000,192.000,247.0
Superplasticizer,1030.0,6.204660,5.973841,0.00,0.000,6.400,10.200,32.2
Coarse Aggregate,1030.0,972.918932,77.753954,801.00,932.000,968.000,1029.400,1145.0
Fine Aggregate,1030.0,773.580485,80.175980,594.00,730.950,779.500,824.000,992.6
Age,1030.0,45.662136,63.169912,1.00,7.000,28.000,56.000,365.0
Strength,1030.0,35.817961,16.705742,2.33,23.710,34.445,46.135,82.6


### Check the null or missing value in the data

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Cement              1030 non-null   float64
 1   Blast Furnace Slag  1030 non-null   float64
 2   Fly Ash             1030 non-null   float64
 3   Water               1030 non-null   float64
 4   Superplasticizer    1030 non-null   float64
 5   Coarse Aggregate    1030 non-null   float64
 6   Fine Aggregate      1030 non-null   float64
 7   Age                 1030 non-null   int64  
 8   Strength            1030 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 72.6 KB


In [8]:
df.isnull().sum()
# there is no any null or missing values

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Prepare the data for features as input and target as output

In [9]:
features = df[df.columns.to_list()[:-1]]
target = df['Strength']

In [10]:
features.shape[1]

8

In [11]:
# Let's Normalize the Data
features_norm = (features-features.mean())/features.std()

## Let's define regression model function
### With three hidden layer of 10 nodes, and a ReLU activation function
### Using the adam optimizer and the mean squared error  as the loss function.

In [12]:
def regression_model():
  model = Sequential()

  # three hidden layers with 10 nodes
  model.add(Dense(10, activation='relu', input_shape = (features_norm.shape[1],)))
  model.add(Dense(10, activation='relu'))
  model.add(Dense(10, activation='relu'))

  # one output layer
  model.add(Dense(1))

  model.compile(optimizer='adam', loss='mean_squared_error')
  return model

## Split the data into training and testing sets using train_test_split from sklearn with 30% testing size

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features_norm, target, test_size = 0.3, random_state=4)

## Training the model on training data X_train and y_train using 50 epochs

In [14]:
# let's call the function
model = regression_model()

In [15]:
# Train the model on training
model.fit(X_train, y_train, epochs = 50, verbose=2)

Epoch 1/50
23/23 - 2s - loss: 1565.2217 - 2s/epoch - 75ms/step
Epoch 2/50
23/23 - 0s - loss: 1534.8315 - 76ms/epoch - 3ms/step
Epoch 3/50
23/23 - 0s - loss: 1495.2812 - 88ms/epoch - 4ms/step
Epoch 4/50
23/23 - 0s - loss: 1437.9948 - 58ms/epoch - 3ms/step
Epoch 5/50
23/23 - 0s - loss: 1352.2924 - 52ms/epoch - 2ms/step
Epoch 6/50
23/23 - 0s - loss: 1220.6694 - 51ms/epoch - 2ms/step
Epoch 7/50
23/23 - 0s - loss: 1035.3363 - 52ms/epoch - 2ms/step
Epoch 8/50
23/23 - 0s - loss: 806.4414 - 51ms/epoch - 2ms/step
Epoch 9/50
23/23 - 0s - loss: 580.1891 - 52ms/epoch - 2ms/step
Epoch 10/50
23/23 - 0s - loss: 403.8063 - 56ms/epoch - 2ms/step
Epoch 11/50
23/23 - 0s - loss: 305.1059 - 52ms/epoch - 2ms/step
Epoch 12/50
23/23 - 0s - loss: 259.6877 - 67ms/epoch - 3ms/step
Epoch 13/50
23/23 - 0s - loss: 237.9204 - 133ms/epoch - 6ms/step
Epoch 14/50
23/23 - 0s - loss: 224.4091 - 86ms/epoch - 4ms/step
Epoch 15/50
23/23 - 0s - loss: 215.3683 - 100ms/epoch - 4ms/step
Epoch 16/50
23/23 - 0s - loss: 208.2311 -

## Predict the model on testing data X_test

In [16]:
# predict the model on test data
predictions = model.predict(X_test)

10/10 [==============================] - 0s 5ms/step


## Evaluate mean_square error on predicted data and y_test data using scikit learn

In [17]:
mse = mean_squared_error(predictions, y_test)
print(f"The mean squared error is: {mse}")

The mean squared error is: 146.48424760052993


In [ ]:
# list of value of 50 mean_squared error, counted by re running the session after random_state set to 4
mse_value = []
for i in range(0,50):
    model.fit(X_train, y_train, epochs = 50, verbose=2)
    predictions = model.predict(X_test)
    mse = mean_squared_error(predictions, y_test)
    mse_value.append(mse)

In [19]:
# mean and the standard deviation of the mean squared errors
import numpy as np
mean_mse = np.mean(mse_value)
SD_mse = np.std(mse_value)
print(f"The mean and standard deviation of mean square errors is {mean_mse} and {SD_mse}")

The mean and standard deviation of mean square errors is 38.3806777072178 and 9.221095949645797
